# Compute summary statistics for the daily sea ice index.

# From the CSV files determine the day of maximum and minimum extent for each
# month and how that month's max and min ranks with all other months

The input data format is just a date and extent for each day we have data.
```
Year, Month, Day,     Extent,    Missing, Source Data
YYYY,    MM,  DD, 10^6 sq km, 10^6 sq km, Source data product web site: http://nsidc.org/d....
1978,    10,  26,     10.231,      0.000, ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051....
1978,    10,  28,     10.420,      0.000, ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051....
1978,    10,  30,     10.557,      0.000, ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051....
....
```


Start by downloading the daily sea ice extent data from NSIDC's website.

In [1]:
!mkdir -p ../data
!wget -P ../data -qN ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02135/north/daily/data/NH_seaice_extent_final.csv 
!wget -P ../data -qN ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02135/north/daily/data/NH_seaice_extent_nrt.csv
!wget -P ../data -qN ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02135/south/daily/data/SH_seaice_extent_final.csv
!wget -P ../data -qN ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02135/south/daily/data/SH_seaice_extent_nrt.csv


In [2]:
import datetime as dt
import numpy as np
import os
import pandas as pd
from pandas import ExcelWriter


code to read the CSV files.

In [3]:

def parse_the_date(year, mm, dd):
    return dt.date(int(year), int(mm), int(dd))

def slurp_csv(filename):
    data = pd.read_csv(filename, header = None, skiprows=2,
                       names=["year", "mm", "dd", "extent", "missing", "source"],
                       parse_dates={'date':['year', 'mm', 'dd']},
                       date_parser=parse_the_date, index_col='date')
    data = data.drop('missing', axis=1)
    return data

def read_a_hemisphere(hemisphere):
    the_dir = '../data'
    final_prod_filename = os.path.join(the_dir, '{hemi}H_seaice_extent_final.csv'.format(hemi=hemisphere[0:1].upper()))
    nrt_prod_filename = os.path.join(the_dir, '{hemi}H_seaice_extent_nrt.csv'.format(hemi=hemisphere[0:1].upper()))

    final = slurp_csv(final_prod_filename)
    nrt = slurp_csv(nrt_prod_filename)
    all_data = pd.concat([final, nrt])
    return all_data



 Read CSV data

In [4]:
north = read_a_hemisphere('north')
south = read_a_hemisphere('south')
south.head()

,extent,source
date,,
1978-10-26,17.634,ftp://sidads.colorado.edu/pub/DATASETS/nsidc0...
1978-10-28,17.815,ftp://sidads.colorado.edu/pub/DATASETS/nsidc0...
1978-10-30,17.671,ftp://sidads.colorado.edu/pub/DATASETS/nsidc0...
1978-11-01,17.534,ftp://sidads.colorado.edu/pub/DATASETS/nsidc0...
1978-11-03,17.493,ftp://sidads.colorado.edu/pub/DATASETS/nsidc0...


Add columns for year and month: We have do this because when we read the CSV file
we converted the existing year/month/day columns into a python datetime object.
also drop the source because we don't care where the data came from (near real time or production)

In [5]:
def add_year_month_columns(df):
    a = df.copy()
    a = a.drop('source',1)
    a = a.reset_index()
    a['year'] = pd.to_datetime(a.date).dt.year
    a['month'] = pd.to_datetime(a.date).dt.month
    a = a.set_index('date')
    return a

In [6]:
north = add_year_month_columns(north)
south = add_year_month_columns(south)

In [7]:
north.head()

,extent,year,month
date,,,
1978-10-26,10.231,1978,10
1978-10-28,10.420,1978,10
1978-10-30,10.557,1978,10
1978-11-01,10.670,1978,11
1978-11-03,10.787,1978,11


In [8]:
south.head()

,extent,year,month
date,,,
1978-10-26,17.634,1978,10
1978-10-28,17.815,1978,10
1978-10-30,17.671,1978,10
1978-11-01,17.534,1978,11
1978-11-03,17.493,1978,11


Add 5 day rolling mean to the timesereis.

In [9]:
def add_rolling_mean(df, window=5, min_periods=2):
    copy = df.copy()
    # create an empty ts to align our extent data with
    ts = pd.Series(np.nan, index=pd.date_range('1978-10-25', dt.date.today().strftime('%Y-%m-%d')))
    copy.index = copy.index.to_datetime()
    copy = df.align(ts, axis=0, join='right')[0]
    df['5day-Avg'] = pd.rolling_mean(copy['extent'], window=5, min_periods=2)
    return df

Want date back in the columns

In [10]:
north = add_rolling_mean(north)
south = add_rolling_mean(south)

In [11]:
north.head(1)

,extent,year,month,5day-Avg
date,,,,
1978-10-26,10.231,1978,10,NaN


In [12]:
north = north.reset_index()
south = south.reset_index()
north.head(1)


,date,extent,year,month,5day-Avg
0,1978-10-26,10.231,1978,10,NaN


Use a groupby to compute the row locations that represent the minimum and
maximum extent and grab those rows into new variables.  AKA: Filter out everything
but the minimum/maximum extent for each month and year

In [13]:
def select_min_and_max_variable_rows_by_year_and_month(df, variable):
    min_groups = df.loc[df.groupby(['year','month'])[variable].idxmin()][['date', variable, 'year', 'month']]
    max_groups = df.loc[df.groupby(['year','month'])[variable].idxmax()][['date', variable, 'year', 'month']]
    return {'min': min_groups,  'max': max_groups}

create dictionaries of max and min values for each hemisphere and for daily and rolling-mean

In [14]:
n = select_min_and_max_variable_rows_by_year_and_month(north, 'extent')
navg = select_min_and_max_variable_rows_by_year_and_month(north, '5day-Avg')
s = select_min_and_max_variable_rows_by_year_and_month(south, 'extent')
savg = select_min_and_max_variable_rows_by_year_and_month(south, '5day-Avg')


show that we have actually selected different data for daily and 5-average data

In [15]:
n['max'][3:5]

,date,extent,year,month
48,1979-01-30,15.912,1979,1
61,1979-02-25,16.579,1979,2


In [16]:
navg['max'][3:5]

,date,5day-Avg,year,month
48,1979-01-30,15.795333,1979,1
62,1979-02-27,16.515000,1979,2


In [17]:
def add_rank(df, rank_by, ascending):
    df['rank'] = df.groupby('month')[rank_by].rank(ascending=ascending)
    return df

add rank column for each month and hemsiphere's max and min:

In [18]:
n['max'] = add_rank(n['max'], 'extent', ascending=False)
n['min'] = add_rank(n['min'], 'extent', ascending=True)
s['max'] = add_rank(s['max'], 'extent', ascending=False)
s['min'] = add_rank(s['min'], 'extent', ascending=True)

navg['max'] = add_rank(navg['max'], '5day-Avg', ascending=False)
navg['min'] = add_rank(navg['min'], '5day-Avg', ascending=True)
savg['max'] = add_rank(savg['max'], '5day-Avg', ascending=False)
savg['min'] = add_rank(savg['min'], '5day-Avg', ascending=True)



In [19]:
def do_annual_min_max_ranking(df, field):
    min_index = df.groupby(['year'])[field].idxmin()
    max_index = df.groupby(['year'])[field].idxmax()
    mindata = df.loc[min_index][['date', field]]
    mindata['rank'] = mindata[field].rank(ascending=True)
    maxdata = df.loc[max_index][['date', field]]
    maxdata['rank'] = maxdata[field].rank(ascending=False)

    mindata = mindata.set_index(pd.to_datetime(mindata.date).dt.year)
    maxdata = maxdata.set_index(pd.to_datetime(maxdata.date).dt.year)

    maxdata = maxdata.add_prefix('max_')
    mindata = mindata.add_prefix('min_')

    data = pd.concat([mindata, maxdata], axis=1)
    return data
    


It is also desired that we have Annual min/max rank data so revisit the north and south

In [20]:
north_annual_by_day = do_annual_min_max_ranking(north, 'extent')
north_annual_averaged = do_annual_min_max_ranking(north, '5day-Avg')

In [21]:
south_annual_by_day = do_annual_min_max_ranking(south, 'extent')
south_annual_averaged = do_annual_min_max_ranking(south, '5day-Avg')

In [22]:
south_annual_averaged.head(3)

,min_date,min_5day-Avg,min_rank,max_date,max_5day-Avg,max_rank
1978,1978-12-31,7.596000,38,1978-10-28,17.7245,37
1979,1979-02-19,2.928333,25,1979-09-15,18.3230,31
1980,1980-02-26,2.574000,7,1980-09-25,19.0470,11


Write out the data frames in a nice format

In [23]:
a = navg['min'].copy()
    

In [24]:
a.columns

Index([u'date', u'5day-Avg', u'year', u'month', u'rank'], dtype='object')

In [25]:
a.set_index(['year', 'month']).unstack('year')

date                                                              \
year         1978        1979        1980        1981        1982        1983   
month                                                                           
1             NaN  1979-01-02  1980-01-01  1981-01-01  1982-01-02  1983-01-01   
2             NaN  1979-02-07  1980-02-02  1981-02-14  1982-02-01  1983-02-02   
3             NaN  1979-03-31  1980-03-31  1981-03-30  1982-03-27  1983-03-30   
4             NaN  1979-04-30  1980-04-30  1981-04-29  1982-04-30  1983-04-29   
5             NaN  1979-05-30  1980-05-30  1981-05-31  1982-05-30  1983-05-31   
6             NaN  1979-06-29  1980-06-29  1981-06-30  1982-06-29  1983-06-30   
7             NaN  1979-07-31  1980-07-31  1981-07-30  1982-07-31  1983-07-30   
8             NaN  1979-08-30  1980-08-28  1981-08-31  1982-08-30  1983-08-31   
9             NaN  1979-09-21  1980-09-05  1981-09-14  1982-09-13  1983-09-10   
10     1978-10-28  1979-10-01  1980-10-01  1981-10-02  1982-10-01  1983-10-02   
11     1978-11-01  1979-11-02  1980-11-02  1981-11-01  1982-11-02  1983-11-01   
12     1978-12-03  1979-12-02  1980-12-02  1981-12-01  1982-12-02  1983-12-01   

                                                      ...  rank            \
year         1984        1985        1986        1987 ...  2006 2007 2008   
month                                                 ...                   
1      1984-01-02  1985-01-02  1986-01-01  1987-01-02 ...     7    6    8   
2      1984-02-01  1985-02-03  1986-02-02  1987-02-01 ...     1    8   12   
3      1984-03-30  1985-03-11  1986-03-30  1987-03-27 ...     1    3   14   
4      1984-04-29  1985-04-30  1986-04-29  1987-04-30 ...     1    3   11   
5      1984-05-31  1985-05-30  1986-05-31  1987-05-30 ...     1    7    8   
6      1984-06-30  1985-06-29  1986-06-30  1987-06-29 ...     5    6    9   
7      1984-07-30  1985-07-31  1986-07-30  1987-07-31 ...    10    2    9   
8      1984-08-31  1985-08-30  1986-08-31  1987-08-31 ...    11    2    4   
9      1984-09-16  1985-09-11  1986-09-10  1987-09-05 ...    12    2    4   
10     1984-10-02  1985-10-01  1986-10-02  1987-10-01 ...    10    2    4   
11     1984-11-01  1985-11-02  1986-11-01  1987-11-01 ...     7    2   13   
12     1984-12-01  1985-12-02  1986-12-01  1987-12-01 ...     1    4   11   

                                          
year  2009 2010 2011 2012 2013 2014 2015  
month                                     
1        4   10    1   11    3    2    5  
2       11    5    2    3    9    7    6  
3        9    7    5   12   10    6    2  
4       18   19    7   16    8    6    4  
5       12    2    3   11   13    5   31  
6       11    1    3    2    7    4  NaN  
7        6    4    3    1    5    7  NaN  
8        7    5    3    1    6    8  NaN  
9        8    5    3    1    7    6  NaN  
10       7    5    3    1    8    6  NaN  
11       5    3    4    1    6    8  NaN  
12       6    2    5    3    8    7  NaN  

[12 rows x 114 columns]

In [26]:
import calendar
month_names = [calendar.month_name[x] for x in range(1,13)]

def swap_column_level_and_sort(df):
    df.columns = df.columns.swaplevel(1,0)
    df = df.sortlevel(0, axis=1)
    return df

# set index to year and month and then broadcast month back across the columns.
# next swap and sort so that you have the data grouped under the month.
def prepare_for_csv(df):
    df = df.set_index(['year','month']).unstack('month')
    df = swap_column_level_and_sort(df)
    df.columns = df.columns.set_levels(month_names, level=0)
    return df


def write_to_xls(df_list, writer, is_monthly=True):
    for df, sheet in df_list:
        if is_monthly:
            df = prepare_for_csv(df)
        df.to_excel(writer,'{sheet}'.format(sheet=sheet), float_format="%.3f")


writer = ExcelWriter('../output/Sea_Ice_Statistics.xls')

monthly_dataframelist =[(navg['min'], 'Northern 5day Min'),
                        (navg['max'], 'Northern 5day Max'),
                        (savg['min'], 'Southern 5day Min'),
                        (savg['max'], 'Southern 5day Max'),
                        (n['min'], 'Northern Daily Min'),
                        (n['max'], 'Northern Daily Max'),
                        (s['min'], 'Southern Daily Min'),
                        (s['max'], 'Southern Daily Max')]

annual_dataframelist = [(north_annual_averaged, 'North Annual 5day-avg'),
                        (north_annual_by_day, 'North Annual Daily'),
                        (south_annual_averaged, 'South Annual 5day-avg'),
                        (south_annual_by_day, 'South Annual Daily')]

write_to_xls(monthly_dataframelist, writer, is_monthly=True)
write_to_xls(annual_dataframelist, writer, is_monthly=False)

writer.save()

clean up your csv files

In [27]:
!cd ../data ; rm -f NH_seaice_extent_final.csv NH_seaice_extent_nrt.csv SH_seaice_extent_final.csv SH_seaice_extent_nrt.csv